In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df2 = pd.read_csv('HomicideActuals1987-2022.csv', low_memory=False)


df2.info()


In [ ]:
df2.head()

In [ ]:
df2['Inc YR'].unique()

In [ ]:
df2.isnull().sum()

In [ ]:
df2['Inc YR'].value_counts()

In [ ]:
df2.dtypes

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
!head 'HomicideActuals1987-2022.csv'

In [ ]:
df2.loc[df2.duplicated(keep=False)]

In [ ]:
print(len(df2))
df2 = df2.drop_duplicates()
print(len(df2))

# Creating filtered  since 2018 dataset called df

In [ ]:
df = df2[df2['Inc YR'] > 2017].copy()
df = df.reset_index(drop=True)
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df.nunique()

In [ ]:
df['Inc YR'].unique()

# Renaming some columns

In [ ]:
# Rename our columns
df = df.rename(columns={'CO': 'County_Codes', 'NCIC':'NCIC_Agency_Codes',
                        'BCS': 'BCS_number', 'vict num': 'Victim_number',
                        'Rpt MO' : 'Rpt_mo', 'Rpt YR': 'Rpt_YR', 
                        'Tot vic': 'Total_victims', 'Tot susp':'Total_suspects',
                        'V sex': 'Gender', 'V race': 'Victim_Race', 'V age': 'Victim_Age', 
                        'Crm Stat': 'Crime_status','Inc MO':'Incident_Month',
                        'Inc day': 'Inc_day', 'Inc YR': 'Inc_YR', 'Week day': 'Week_day',
                        'death YR': "death_YR", 'Loc': 'Location', 'PE 1': 'Precipitating_event', "Spec Circ 1": 'Spec_Circ_1'})




## Examining Missing Values

In [ ]:
missing_percentages = df.isna().sum().sort_values(ascending=False) / len(df)
missing_percentages

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numeric_df = df.select_dtypes(include=numerics)
len(numeric_df.columns)

In [ ]:
missing_percentages[missing_percentages != 0].plot(kind='barh')

# Initial - Filling in missing values

Recoded variables per Data documentation:
    na values = 'Value not reported'

In [ ]:
df.fillna('Value not reported', inplace=True)

In [ ]:
#no duplicates detected
df.loc[df.duplicated(keep=False)]

Recoding the gender variable as:

In [ ]:
df['Gender'].replace(0, 'Unknown', inplace=True)
df['Gender'].replace(1, 'Male', inplace=True)
df['Gender'].replace(2, 'Female', inplace=True)



In [ ]:
df['Gender'].unique()

In [ ]:
df['Gender'] = df['Gender'].astype(str)


In [ ]:
df.columns

In [ ]:
features = ['County_Codes', 'NCIC_Agency_Codes', 'BCS_number', 'Victim_number',
       'Rpt_mo', 'Rpt_YR', 'Total_victims', 'Total_suspects', 'Gender',
       'Victim_Race', 'Victim_Age', 'Crime_status', 'VO_1', 'VO_2', 'VO_3',
       'VO_4', 'Incident_Month', 'Inc_day', 'Inc_YR', 'Week_day', 'death_YR',
       'Weap', 'Location', 'Precipitating_event', 'Spec_Circ_1']

for feature in features:
    null_check = pd.isnull(df[feature])
    print(df[null_check])

## Recoding the Age variable

In [ ]:
df['Victim_Age'].unique()

In [ ]:

df['Victim_Age'].replace('BB', 'One week to 12 months', inplace=True)
df['Victim_Age'].replace('NB', 'Birth to one week', inplace=True)
df['Victim_Age'].replace('99', '99 or over', inplace=True)
df['Victim_Age'].replace('0', 'Unknown', inplace=True)

In [ ]:
df["Victim_Age"].unique()

# Cleaning the NCIC Agency Codes

In [ ]:
df['NCIC_Agency_Codes'].unique()

In [ ]:
df['NCIC_Agency_Codes'].isna().sum()

In [ ]:
df[df['NCIC_Agency_Codes'] == 'Value not reported']

# Cleaning BCS number

In [ ]:
df['BCS_number'].isna().sum()

In [ ]:
df['BCS_number'].unique()

In [ ]:
df['BCS'] = df['BCS_number']

In [ ]:
df['BCS'].replace('Value not reported', 99999, inplace=True)

In [ ]:
df['BCS'] = df['BCS'].astype(int)

In [ ]:
df['BCS'].unique()

In [ ]:
df.isna().sum()

In [ ]:
df.nunique()

# Cleaning Race variable

In [ ]:
df['Victim_Race'].unique()

In [ ]:
df['Victim_Race'].replace(['H', 'X', 'W', 'B', 'A', 'O', 'U', 'I', 'C', 'S', 'D', 'P', 'F', 'V', 'Z', 'K', 'J', 'L', 'G'], 
                   ['Hispanic', 'Unkown', 'White', 'Black', 'Other Asian', 'Other', 'Hawaiian', 'American Indian', 'Chinese','Samoan',
                   'Cambodian', 'Pacific Islander', 'Filipino', 'Vietnamese', 'Asian Indian', 'Korean','Japanese','Laotian', 'Guamanian'],
                   inplace=True)



In [ ]:
df['Victim_Race'].unique()

# Crime Status
## only includes Actual/willful homicide

In [ ]:
df['Crime_status'].unique()

In [ ]:
df['Crime_status'].replace(1,'Actual/Willful Homicide' , inplace=True)

# Recoding Victim Offender Relationship with full text description per data documentation

In [ ]:
VO_1 = [30.0, 50.0, 21.0, 40.0, 2.0, 23.0, 45.0, 20.0, 1.0, 28.0, 5.0,
       16.0, 9.0, 4.0, 6.0, 27.0, 26.0, 11.0, 8.0, 7.0, 10.0, 22.0, 12.0,
       25.0, 29.0, 14.0, 15.0, 24.0, 13.0, 3.0]

VO_1.sort(key = float)
VO_1

In [ ]:
VO_2 = ['Husband',
        'Wife',
        'Common-law husband',
        'Common-law wife',
        'Mother',
        'Father',
        'Son',
        'Daughter',
        'Brother',
        'Sister',
        'In-law',
        'Stepfather',
        'Stepmother',
        'Stepson',
        'Stepdaughter',
        'Other family',
        'Neighbor',
        'Acquaintance',
       ' Boyfriend/Ex-Boyfriend',
       ' Girlfriend/Ex-Girlfriend',
        'Ex-husband',
        'Ex-wife',
        'Employer',
        'Employee',
        'Friend',
        'Homosexual relationship',
        'Other known to Victim',
        'Stranger',
        'Gang member (Beginning,1992)',
        'Unable to determine relationship']





In [ ]:
df['Victim_Offender_Relationship_1'] = df['VO_1']


In [ ]:
df['Victim_Offender_Relationship_1'].replace(VO_1,
                   VO_2,
                   inplace=True)


In [ ]:
df['Victim_Offender_Relationship_1'].unique()

# Recoding Week Day variable from code to full text

In [ ]:
df['Week_day'].unique()

In [ ]:
df['Week_day'].replace([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0], 
                       ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday', 'Saturday'], 
                        inplace=True)

In [ ]:
df['Week_day'].unique()

#  Incident Month
will keep as is for now

In [ ]:
df['Incident_Month'].unique()

In [ ]:
df['Incident_Month'].dtype

# Recoding Precipitaing event 

In [ ]:
df['Precipitating_event'].unique()

In [ ]:
df['Precipitating_event'].dtype

In [ ]:
Perc_event_codes = [99,  3, 26, 62, 63, 46, 45, 18, 47,  7, 70, 43, 44, 48, 42, 60, 61, 40,  5,  2,  9, 19, 10, 41,  6, 17]
Perc_event_codes



In [ ]:
Perc_event_codes.sort(key = int)
Perc_event_codes

In [ ]:
Perc_events = ['Rape',
        'Robbery',
        'Burglary',
        'Larceny',
        'Motor vehicle theft',
        'Arson',
        'Prostitution and commercial vice (i.e., prostitute killed by pimp during argument over money)',
        'Other sex offense (sodomy, etc.)',
        'Narcotic drug laws (not overdose)',
        'Gambling',
        'Other not specified (kidnapping, murder for hire)',
        'Lover''s triangle',
        'Child killed by babysitter',
        'Brawl due to influence of alcohol (i.e., argument while intoxicated)',
        'Brawl due to influence of narcotics (i.e., argument while under the influence)',
        'Argument over money or property (revenge over money or property)',
        'Other arguments (revenge)',
        'Execution killing(s)',
        'Gang killing(s) (street gangs, motorcycle gangs)',
        'Institutional killing(s) (jail, prison, state hospital)',
        'Other - non-felony (mercy killing, suicide pacts, mentally deranged offender)',
        'Child abuse',
        'Domestic Violence (1992) (Revised June 2023)',
        'Drive-by Shooting (unspecified) (1996)',
        'All - suspected felony type',
        'All instances where facts provided do not permit determination of circumstances (skeletal remains)',
        ]

In [ ]:
Perc_events

In [ ]:
df['Precipitating_event'].replace(Perc_event_codes,
                   Perc_events,
                   inplace=True)

In [ ]:
df['Precipitating_event'].unique()

# Creating incident date variable as datetime dtype

In [ ]:
df['date'] = pd.to_datetime(dict(year=df.Inc_YR, month=df.Incident_Month, day=df.Inc_day))


In [ ]:
df.info()

In [ ]:
df.head()

# Getting NCIC Codes for location mapping
## will only use county level analysis here

In [ ]:
NCIC = pd.read_excel("NCIC Code Jurisdiction List_04242023.xlsx")
NCIC.head()

In [ ]:
NCIC.info()

In [ ]:
NCIC.sort_values(by='CntyCode', ascending = True, inplace = True)
NCIC.head()

In [ ]:
county_codes = NCIC[['CntyCode','County']]

In [ ]:
county_codes.drop_duplicates()

In [ ]:
county_codes.reset_index()

In [ ]:
county_codes.info()

In [ ]:
county_codes = county_codes.drop_duplicates()
county_codes = county_codes.reset_index(drop=True)
county_codes

In [ ]:
# Rename our columns
county_codes = county_codes.rename(columns={'CntyCode' : 'County_Codes'})



In [ ]:
df = pd.merge(df, county_codes, how='outer', on='County_Codes')

In [ ]:
df.info()

In [ ]:
df.replace({pd.NaT: None}, inplace=True)


In [ ]:
homicide_df = df[['Gender','Victim_Race', 'Victim_Age', 'Incident_Month', 'Inc_YR', 'Week_day',
       'Precipitating_event','Victim_Offender_Relationship_1', 'date', 'County']]

In [ ]:
homicide_df = homicide_df[homicide_df['Inc_YR'].notnull()].reset_index(drop=True)

In [ ]:
homicide_df.info()

# This is the final csv file we will use with all selected observations and columns

In [ ]:
homicide_df.head()

In [ ]:
homicide_df.to_csv('homicide.csv')